# Generative Performance
## Fitting models to player 2 (Trustee)

In [3]:
import sys
sys.path.append('/Users/bryangonzalez/Documents/Dartmouth/Research/TheoryOfMind_Bx_experiment/Code/models/')
sys.path.append('/Users/bryangonzalez/Documents/Dartmouth/Research/TheoryOfMind_Bx_experiment/Code/')
import pandas as pd
import numpy as np
from scipy.optimize import least_squares
import p3_cost_funcs
import comp_models
from utils import  folk_params, compute_aic, compute_bic


dat = pd.read_csv('../data/bx_data.csv', index_col=0)

results = pd.DataFrame(columns=['p2', 'p3', 'p2_strat', 'model', 'param1', 'param2', 'sse', 'aic', 'bic', 'fit_method'])



niters = 100


## Fit models to p3

In [2]:
n_states = 8

for turk in dat.turker_id.unique():
    game = dat.loc[dat.turker_id == turk].reset_index(drop=True)

    #Fit mental state space mode
    model = 'mental state space'
    print('Fitting '+model+' model to turker: '+str(turk)+'.')
    fitIters = np.zeros([niters, 5])
    for i in range(niters):
        param0 = [np.random.uniform(), np.random.uniform(low=.001)]
        fitIters[i,0:2] = [param0[0], param0[1]]
        result_lsq = least_squares(fun=p3_cost_funcs.mss_costfun, x0=param0,
                                   args=(game, n_states), bounds=([0,.001], [1,1]), diff_step=.05)

        [param1, param2] = result_lsq.x
        cost = result_lsq.cost
        fitIters[i, 2:5] = [param1, param2, cost]
    cost_selected = np.min(fitIters[:, 4])
    alpha = fitIters[fitIters[:,4]== cost_selected, 2][0]
    tau = fitIters[fitIters[:,4]== cost_selected, 3][0]
    sse = cost_selected* 2
    results = results.append(pd.DataFrame(
        [[game['trustee'].values[0], game['turker_id'].values[0], game['p2_strat'].values[0],
          model, alpha, tau, sse, compute_aic(len(game), sse, 2), compute_bic(len(game), sse, 2),
          'least_squares_'+str(niters)+'iters'
          ]],
        columns=results.columns
    ))

    if turk%5 == 0:
        results.to_csv('results/lstsqrs_fit2_p3_100iters_mss.csv')
'''
    #Fit Rescorla Wagner
    model = 'rescorla-wagner'
    print('Fitting '+model+' model turker: '+str(turk)+'.')
    fitIters = np.zeros([niters, 5])
    for i in range(niters):
        param0 = np.random.uniform()
        fitIters[i,0:2] = [param0, np.nan]
        result_lsq = least_squares(fun=p3_cost_funcs.rw_costfun, x0=param0, args=(game, ), bounds=(0,1), diff_step=.05)

        param1 = result_lsq.x
        cost = result_lsq.cost
        fitIters[i, 2:5] = [param1, np.nan, cost]
    cost_selected = np.min(fitIters[:, 4])
    alpha = fitIters[fitIters[:,4]== cost_selected, 2][0]
    sse = cost_selected* 2
    results = results.append(pd.DataFrame(
        [[game['trustee'].values[0], game['turker_id'].values[0], game['p2_strat'].values[0],
          model, alpha,np.nan, sse, compute_aic(len(game), sse, 1), compute_bic(len(game), sse, 1),
          'least_squares_'+str(niters)+'iters'
          ]],
        columns=results.columns
    ))


    #Fit Inverse Motivation Model
    model = 'inverse motive'
    print('Fitting '+model+' model to turker: '+str(turk)+'.')
    fitIters = np.zeros([niters, 5])
    for i in range(niters):
        param0 = [np.random.uniform(), np.random.uniform(low=.001)]
        fitIters[i,0:2] = [param0[0], param0[1]]
        result_lsq = least_squares(fun=p3_cost_funcs.imm_costfun, x0=param0,
                                   args=(game, folk_params ), bounds=([0,.001], [1,1]), diff_step=.05)

        [param1, param2] = result_lsq.x
        cost = result_lsq.cost
        fitIters[i, 2:5] = [param1, param2, cost]
    cost_selected = np.min(fitIters[:, 4])
    alpha = fitIters[fitIters[:,4]== cost_selected, 2][0]
    tau = fitIters[fitIters[:,4]== cost_selected, 3][0]
    sse = cost_selected* 2
    results = results.append(pd.DataFrame(
        [[game['trustee'].values[0], game['turker_id'].values[0], game['p2_strat'].values[0],
          model, alpha, tau, sse, compute_aic(len(game), sse, 2), compute_bic(len(game), sse, 2),
          'least_squares_'+str(niters)+'iters'
          ]],
        columns=results.columns
    ))


    # "Fit" Greed Model
    model = 'greed'
    print('Fitting '+model+' model to turker: '+str(turk)+'.')
    sse = sum(np.square(p3_cost_funcs.gr_costfun(game)))
    results = results.append(pd.DataFrame(
        [[game['trustee'].values[0], game['turker_id'].values[0], game['p2_strat'].values[0],
          model, np.nan, np.nan, sse, compute_aic(len(game), sse, 0), compute_bic(len(game), sse, 0),
          'least_squares_'+str(niters)+'iters'
          ]],
        columns=results.columns
    ))


    #Fit Guilt-aversion model
    model = 'guilt-aversion'
    print('Fitting '+model+' model to turker: '+str(turk)+'.')
    for i in range(niters):
        param0 = np.random.uniform()*10000
        fitIters[i,0:2] = [param0, np.nan]
        result_lsq = least_squares(fun=p3_cost_funcs.ga_costfun, x0=param0,
                                   args=(game, ), bounds=([0],[10000]))

        param1 = result_lsq.x
        cost = result_lsq.cost
        fitIters[i, 2:5] = [param1, np.nan, cost]
    cost_selected = np.min(fitIters[:, 4])
    theta = fitIters[fitIters[:,4]== cost_selected, 2][0]
    sse = cost_selected* 2
    results = results.append(pd.DataFrame(
        [[game['trustee'].values[0], game['turker_id'].values[0], game['p2_strat'].values[0],
          model, theta, np.nan, sse, compute_aic(len(game), sse, 1), compute_bic(len(game), sse, 1),
          'least_squares_'+str(niters)+'iters'
          ]],
        columns=results.columns
    ))


    #Fit Inequity-aversion model
    model = 'inequity-aversion'
    print('Fitting '+model+' model to turker: '+str(turk)+'.')
    for i in range(niters):
        param0 = np.random.uniform()*10000
        fitIters[i,0:2] = [param0, np.nan]
        result_lsq = least_squares(fun=p3_cost_funcs.ia_costfun, x0=param0,
                                   args=(game, ), bounds=([0],[10000]))

        param1 = result_lsq.x
        cost = result_lsq.cost
        fitIters[i, 2:5] = [param1, np.nan, cost]
    cost_selected = np.min(fitIters[:, 4])
    theta = fitIters[fitIters[:,4]== cost_selected, 2][0]
    sse = cost_selected* 2
    results = results.append(pd.DataFrame(
        [[game['trustee'].values[0], game['turker_id'].values[0], game['p2_strat'].values[0],
          model, theta, np.nan, sse, compute_aic(len(game), sse, 1), compute_bic(len(game), sse, 1),
          'least_squares_'+str(niters)+'iters'
          ]],
        columns=results.columns
    ))


    #Fit Moral Phenotype model
    model = 'moral phenotype'
    print('Fitting '+model+' model to turker: '+str(turk)+'.')
    fitIters = np.zeros([niters, 5])
    for i in range(niters):
        param0 = [np.random.uniform()/2, np.random.uniform()/5-0.1]
        fitIters[i,0:2] = [param0[0], param0[1]]
        result_lsq = least_squares(fun=p3_cost_funcs.mp_costfun, x0=param0, args=(game, ),
                                   bounds=([0,-.1], [.5,.1]))

        [param1, param2] = result_lsq.x
        cost = result_lsq.cost
        fitIters[i, 2:5] = [param1, param2, cost]
    cost_selected = np.min(fitIters[:, 4])
    theyta = fitIters[fitIters[:,4]== cost_selected, 2][0]
    phee = fitIters[fitIters[:,4]== cost_selected, 3][0]
    sse = cost_selected* 2
    results = results.append(pd.DataFrame(
        [[game['trustee'].values[0], game['turker_id'].values[0], game['p2_strat'].values[0],
          model, theyta, phee, sse, compute_aic(len(game), sse, 2), compute_bic(len(game), sse, 2),
          'least_squares_'+str(niters)+'iters'
          ]],
        columns=results.columns
    ))

    if turk%5 == 0:
        results.to_csv('results/lstsqrs_fit2_p3_100iters_mss.csv')
'''

Fitting mental state space model to turker: 1.


/Users/bryangonzalez/Documents/Dartmouth/Research/TheoryOfMind_Bx_experiment/Code/models/p3_cost_funcs.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_predictions = pd.Series(mentalState_space(game, alpha, tau, nStates))
/Users/bryangonzalez/Documents/Dartmouth/Research/TheoryOfMind_Bx_experiment/Code/models/comp_models.py:253: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counterfactuals[theyta][phee] = mp_model_ppsoe(inv=game.inv.iloc[t], mult=game.mult.iloc[t], theta=theyta, phi=phee)


KeyError: -0.1

In [ ]:
results.to_csv('results/lstsqrs_fit2_p3_100iters_mss.csv')